In [ ]:
import os
import gmsh
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from fem_pre_processing import read_mesh
from fem_processing import gaussian_quadrature, matrices_assembly
from fem_pos_processing import graph_results as graph

# `apply_physics()`

In [2]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    for key, cell in mesh_data['cell'].items():
        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []

        # Get the global coordinates of the nodes
        ai = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Get the Gauss points and weights
        gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)
        
        # Adicionar a nova chave 'material' ao dicionário da célula
        for xik in gauss_points: 
            xg_e, yg_e = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, ai, xik)
            cell['stiffness_term'].append(np.eye(2))
            cell['mass_term'].append(1)           
        
            # Adicionar a nova chave 'source' ao dicionário da célula
            cell['source'].append(0)

        # Dictionary with boundary nodes
        cell_data[key]['abc'] = {'type': None, 'conn_idx': None}
    
    return mesh_data

# `create_rectangular_guide()`

In [ ]:
def create_rectangular_guide(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    mesh_data = {}
    type, order = FINITE_ELEMENT

    # Dimensões do guia de onda retangular
    a, b = 8e-2, 4e-2 

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("rectangular_guide")

    # Criar superfície retangular
    TagSurface = gmsh.model.occ.addRectangle(0, 0, 0, a, b)
    gmsh.model.occ.synchronize()
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.model.mesh.generate(dim=2)
    gmsh.model.mesh.setOrder(order)

    # Obter os contornos (curvas, dim=1) de cada superfície
    outDimTags = gmsh.model.getBoundary([(2, TagSurface)], oriented=True, recursive=False)

    # Exibir os TAGs das curvas associadas a cada contorno
    tagList_boundary = [Dimtags[1] for Dimtags in outDimTags]

    # Definindo as curvas de contorno de Dirichlet (dim=1)
    gmsh.model.addPhysicalGroup(dim=1, tags=tagList_boundary, tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])

    # Adicionar grupos físicos para Dim=2 (superfícies)
    gmsh.model.addPhysicalGroup(dim=2, tags=[TagSurface], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_guide_domain_{type}{order}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)
    
    gmsh.finalize()

    return mesh_data

# `waveguide_modes()`

In [ ]:
def waveguide_modes(m_max, n_max, mode_type):
    """
    Calcula os modos TE ou TM para um guia de onda retangular.

    Parâmetros:
    a (float): Dimensão transversal a (em metros).
    b (float): Dimensão transversal b (em metros).
    m_max (int): Número máximo de modos para m.
    n_max (int): Número máximo de modos para n.
    mode_type (str): Tipo de modo, 'TE' ou 'TM'.

    Retorna:
    dict: Dicionário com os modos e seus respectivos kc.
    """
    # Dimensões do guia de onda retangular
    a, b = 8e-2, 4e-2 

    if mode_type not in ['TE', 'TM']:
        raise ValueError("mode_type deve ser 'TE' ou 'TM'")

    modes = {}
    for m in range(m_max + 1):
        for n in range(n_max + 1):
            if mode_type == 'TE' and m == 0 and n == 0:
                continue  # Excluir o modo (0,0) para TE
            if mode_type == 'TM' and (m == 0 or n == 0):
                continue  # Excluir modos onde m=0 ou n=0 para TM

            kc = np.sqrt((m * np.pi / a)**2 + (n * np.pi / b)**2)
            modes[f"{mode_type}_{m}{n}"] = kc

    # Ordenar os autovalores de te_modes e listar os cinco menores
    sorted_te_modes = sorted(modes.items(), key=lambda x: x[1])
    five_smallest_modes = sorted_te_modes[:m_max]

    # Imprimir os cinco menores autovalores dos modos TE
    print(f"\nCinco menores autovalores analíticos dos modos {mode_type}:")
    for mode, kc in five_smallest_modes:
        print(f"{mode}: kc2 = {kc**2:.3f} rad2/m2")
    
    return five_smallest_modes

# `calculate_error()`

In [2]:
def calculate_error(reference_modes, computed_modes):
    """
    Calcula o erro entre os valores de referência (modos analíticos) e os modos computados numericamente.

    Parâmetros:
    reference_modes (list): Lista de tuplas representando os modos analíticos, onde cada tupla contém o nome do modo e o valor de kc.
    computed_modes (list): Lista de valores computados numericamente para os menores kc^2.

    Retorna:
    dict: Um dicionário contendo os erros absolutos para cada modo correspondente.
    """
    # Extrair os valores de kc do dicionário de referência
    reference_values = [kc for _, kc in reference_modes]

    # Verificar se os tamanhos são compatíveis
    if len(reference_values) != len(computed_modes):
        raise ValueError("O número de modos de referência e os modos computados devem ser iguais.")

    # Calcular kc^2 para os valores de referência
    reference_kc_squared = [kc**2 for kc in reference_values]

    # Calcular os erros percentuais
    errors = [abs(ref - comp)/ref * 100 for ref, comp in zip(reference_kc_squared, computed_modes)]

    # Retornar os erros em um dicionário
    errors_dict = {name: error for (name, _), error in zip(reference_modes, errors)}

    for mode, error in errors_dict.items():
        print(f"Modo {mode}: Erro percentual = {error:.4f}")

# `potential_and_gradient()`

In [4]:
def potential_and_gradient(x, y):
    return 0, np.zeros((2,1))

# `get_modes_pattern()`

In [ ]:
def get_modes_pattern(mesh_data, eigenvalues, eigenvectors, modes, type='abs'):
    # Solução aproximada
    for i, eigenvalue in enumerate(eigenvalues):
        # Solução aproximada global
        uh = matrices_assembly.global_potentials_solution(mesh_data, eigenvectors[:, i])

        # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
        xg, yg, conn = graph.structured_data(mesh_data)

        # Para elementos P1 (3 nós por elemento)
        triangulation = Triangulation(xg, yg, conn)

        # Conversão do dicionário de soluções para lista
        if type == 'real':
            uh = np.real(list(uh.values()))
        elif type == 'imag':
            uh = np.imag(list(uh.values()))
        elif type == 'abs':
            uh = np.abs(list(uh.values()))
        else:
            raise ValueError("Tipo de solução inválida. Use 'real', 'imag' ou 'abs'.")
    
        plt.figure(figsize=(8, 6))
        plt.tricontour(triangulation, uh, cmap='viridis')
        plt.title(rf'Solução Aproximada para Autovalor $\lambda_{i}$ = {eigenvalue:.3f}. Modo {modes[i][0]}')
        plt.xlabel(r'$x$')
        plt.ylabel(r'$y$')
        plt.tight_layout()

# `plot_dispersive_curve()`

In [ ]:
def plot_dispersive_curve(eigenvalues):

    # Constante de onda de corte do modo TE10 (1/m)
    kc_10 = np.sqrt(eigenvalues[0])   

    # Varredura de k (# Evita k=0)
    k = np.linspace(1 * kc_10, 4 * kc_10, 500)  

    # Cálculo da dispersão para o modo TE10
    beta = np.sqrt(k**2 - kc_10**2)  

    # Normalização
    k_normalized = k / kc_10
    beta_normalized = beta / k

    # Plota o gráfico de dispersão
    plt.figure(figsize=(8, 6))
    plt.plot(k_normalized, beta_normalized, label="Modo TE10", color="blue")

    # Destaque da faixa de operação monomodo
    plt.axvline(1, color="red", linestyle="--", label="k = k_c10")
    plt.axvline(4, color="green", linestyle="--", label='k = 4k_c10')

    # Configurações do gráfico
    plt.xlabel(r"$k/k_{c10}$", fontsize=12)
    plt.ylabel(r"$\beta/k$", fontsize=12)
    plt.title("Curva de Dispersão do Modo TE10", fontsize=14)
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()  

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.